## Setting

In [1]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn
import timm
from PIL import Image
from tqdm import tqdm
import random
import os

%config Completer.use_jedi = False

In [2]:
def setseed(seednum = 20):
    torch.manual_seed(20)
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20)
    np.random.seed(20)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(20)

In [3]:
setseed(30)

In [4]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
device

device(type='cuda')

## Dataset

In [5]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [6]:
train_set = ds.CIFAR10(root='../data/', train=True, transform=transform, download=True)
val_set = tv.datasets.CIFAR10(root='../data/', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=100, shuffle=False, num_workers=0)

In [8]:
def make_test_list(data_dir):
    
    test_img_list = list()
    
    files = os.listdir(data_dir)
    
    for i in range(10):
        img_file = data_dir + "/" + files[i]
        img = os.listdir(img_file)
        for j in range(200):
            img_path = data_dir + "/" +  files[i] + "/" + img[j]
            
            test_img_list.append(img_path)
    return test_img_list

In [9]:
img_list = make_test_list("../Statistical_Deep_Image_2021/Statistical_Deep_Image")

In [10]:
def make_test_label(label_list):
    
    test_label_list = list()
    
    for i in range(10):
        for j in range(200):
            test_label_list.append(i)
        
    return test_label_list

In [11]:
label = os.listdir("../Statistical_Deep_Image_2021/Statistical_Deep_Image")
img_label_list = make_test_label(label)

In [12]:
class testset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_label_list, transform):
        self.file_list = img_list
        self.labels = img_label_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        label = self.labels[index]
        return img_transformed, label

In [13]:
test_set = testset(img_list = img_list,
                  img_label_list = img_label_list,
                  transform = transform)

In [14]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 100, shuffle=False, num_workers=0)

In [15]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Model

In [16]:
model = timm.create_model('vgg16', pretrained=False, num_classes=10)

In [17]:
model = model.to(device)

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Train

In [22]:
for epoch in tqdm(range(40)):
    
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0) :
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'epoch: {epoch + 1} loss: {running_loss / 500:.3f}')

  2%|▎         | 1/40 [01:06<43:19, 66.65s/it]

epoch: 1 loss: 2.054


  5%|▌         | 2/40 [02:13<42:14, 66.69s/it]

epoch: 2 loss: 1.639


  8%|▊         | 3/40 [03:08<37:56, 61.52s/it]

...

epoch: 38 loss: 0.021


 98%|█████████▊| 39/40 [36:56<01:04, 64.10s/it]

epoch: 39 loss: 0.021


100%|██████████| 40/40 [38:03<00:00, 57.09s/it]

epoch: 40 loss: 0.017


In [23]:
# 모델 가중치 저장
torch.save(model.state_dict(), 'model_weights.pth')

#모델 가중치 불러오기
#model.load_state_dict(torch.load('model_weights.pth'))
#model.eval()

## Test

In [24]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))

100%|██████████| 20/20 [00:01<00:00, 18.24it/s]

Accuracy: 36 %
